In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
data = pd.read_csv("final_opponent_and_team_data.csv")
data.head()

,Date,Team,Opponent,Location,WAB,ADJO,ADJD,EFF,EFG%,TO%,OR%,FTR,2P,3P,Opp EFF,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,Opp 2P,Opp 3P,Opponent_score,Team_score,opp_adj_o,opp_adj_d
0,2024-11-04,Duke,Maine,H,0.1,125.3,95.2,130.6,64.3,17.7,35.5,31.7,24-34,11-29,84.3,39.5,19.0,19.4,31.6,18-43,3-14,62,96,108.9,110.8
1,2024-11-08,Duke,Army,H,0.1,124.7,92.3,141.0,61.3,11.3,43.6,25.4,18-33,17-38,81.8,39.1,18.3,23.3,14.1,13-35,8-29,58,100,107.2,119.0
2,2024-11-12,Duke,Kentucky,N,-0.2,106.8,86.4,95.7,42.3,9.3,25.0,23.9,24-47,4-24,102.3,47.6,14.6,23.3,38.1,15-38,10-25,77,72,119.6,88.1
3,2024-11-16,Duke,Wofford,H,-0.1,124.7,56.9,133.7,61.3,15.5,45.2,17.7,14-24,16-38,54.4,28.9,29.5,29.3,5.3,9-24,5-33,35,86,69.8,115.5
4,2024-11-22,Duke,Arizona,A,0.6,111.7,75.9,101.9,50.0,20.7,35.1,21.3,17-36,9-25,81.2,45.3,22.2,16.7,20.8,15-30,6-23,55,69,98.9,90.4


In [3]:
data.dtypes

Date               object
Team               object
Opponent           object
Location           object
WAB               float64
ADJO              float64
ADJD              float64
EFF               float64
EFG%              float64
TO%               float64
OR%               float64
FTR               float64
2P                 object
3P                 object
Opp EFF           float64
Opp EFG%          float64
Opp TO%           float64
Opp OR%           float64
Opp FTR           float64
Opp 2P             object
Opp 3P             object
Opponent_score      int64
Team_score          int64
opp_adj_o         float64
opp_adj_d         float64
dtype: object

In [4]:
data.isnull().sum()

Date              0
Team              0
Opponent          0
Location          0
WAB               0
ADJO              0
ADJD              0
EFF               0
EFG%              0
TO%               0
OR%               0
FTR               0
2P                0
3P                0
Opp EFF           0
Opp EFG%          0
Opp TO%           0
Opp OR%           0
Opp FTR           0
Opp 2P            0
Opp 3P            0
Opponent_score    0
Team_score        0
opp_adj_o         0
opp_adj_d         0
dtype: int64

In [5]:
data[data.isnull().any(axis=1)]

,Date,Team,Opponent,Location,WAB,ADJO,ADJD,EFF,EFG%,TO%,OR%,FTR,2P,3P,Opp EFF,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,Opp 2P,Opp 3P,Opponent_score,Team_score,opp_adj_o,opp_adj_d


In [6]:
data = data.dropna()

In [7]:
def preprocess(data):
    data_clean = data.dropna()
    columns_to_convert = [
        "ADJO",
        "ADJD",
        "EFG%",
        "TO%",
        "OR%",
        "FTR",
        "Opp EFG%",
        "Opp TO%",
        "Opp OR%",
        "Opp FTR",
    ]
    for col in columns_to_convert:
        data[col] = pd.to_numeric(data[col], errors="coerce")

    ratio_columns = ["2P", "3P", "Opp 2P", "Opp 3P"]

    # Convert existing ratio columns to numerical percentages
    for column in ratio_columns:
        if column in data.columns:  # Check if the column exists in the dataset
            data[column] = (
                data[column]
                .str.split("-")
                .apply(
                    lambda x: (
                        int(x[0]) / int(x[1]) if len(x) == 2 and int(x[1]) != 0 else 0
                    )
                )
            )
    categorical_columns = ["Location"]
    data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

    data_encoded = data_encoded.apply(
        lambda x: x.astype(int) if x.dtype == "bool" else x
    )

    return data_encoded

In [8]:
preprocessed_data = preprocess(data)

In [9]:
X = preprocessed_data.drop(
    columns=[
        "Team_score",
        "Opponent_score",
        "Team",
        "Opponent",
        "2P",
        "3P",
        "Opp 2P",
        "Opp 3P",
        "WAB",
        "EFF",
        "Opp EFF",
        "Date",
    ]
)
y = preprocessed_data[["Team_score", "Opponent_score"]]

X = X.fillna(0)

In [10]:
X.dtypes

ADJO          float64
ADJD          float64
EFG%          float64
TO%           float64
OR%           float64
FTR           float64
Opp EFG%      float64
Opp TO%       float64
Opp OR%       float64
Opp FTR       float64
opp_adj_o     float64
opp_adj_d     float64
Location_H      int64
Location_N      int64
dtype: object

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [12]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [13]:
model = MLPRegressor(
    hidden_layer_sizes=(64, 32), activation="relu", max_iter=500, random_state=42
)

In [14]:
# Team score
y_train_team = y_train[:, 0]
# Opponent Score
y_train_opponent = y_train[:, 1]

In [15]:
# Perform 5-fold cross-validation for Team_score
cv_scores_team = cross_val_score(
    model, X_train, y_train_team, cv=5, scoring="neg_mean_absolute_error"
)

# Perform 5-fold cross-validation for Opponent_score
cv_scores_opponent = cross_val_score(
    model, X_train, y_train_opponent, cv=5, scoring="neg_mean_absolute_error"
)

# Convert negative MAE scores to positive
cv_scores_team = -cv_scores_team
cv_scores_opponent = -cv_scores_opponent

# Print the cross-validation results
print(
    f"Cross-validated MAE for Team Score: {cv_scores_team.mean()} ± {cv_scores_team.std()}"
)
print(
    f"Cross-validated MAE for Opponent Score: {cv_scores_opponent.mean()} ± {cv_scores_opponent.std()}"
)

Cross-validated MAE for Team Score: 0.5246903033438819 ± 0.06295566592312198
Cross-validated MAE for Opponent Score: 0.429819358529462 ± 0.0436049438476081


In [16]:
# Fit the model on the full training data
model.fit(X_train, y_train)

# Make predictions on the test data
ypred_scaled = model.predict(X_test)

# Inverse transform the predictions for both Team_score and Opponent_score
y_pred = scaler_y.inverse_transform(ypred_scaled)

# Evaluate the model's performance
mae_team = mean_absolute_error(y_test[:, 0], y_pred[:, 0])
mae_opponent = mean_absolute_error(y_test[:, 1], y_pred[:, 1])

mse_team = mean_squared_error(y_test[:, 0], y_pred[:, 0])
mse_opponent = mean_squared_error(y_test[:, 1], y_pred[:, 1])

r2_team = r2_score(y_test[:, 0], y_pred[:, 0])
r2_opponent = r2_score(y_test[:, 1], y_pred[:, 1])

# Print the evaluation metrics
print("\nTeam Score Prediction Metrics:")
print(f"MAE: {mae_team}")
print(f"MSE: {mse_team}")
print(f"R² Score: {r2_team}")

print("\nOpponent Score Prediction Metrics:")
print(f"MAE: {mae_opponent}")
print(f"MSE: {mse_opponent}")
print(f"R² Score: {r2_opponent}")


Team Score Prediction Metrics:
MAE: 79.05576141895446
MSE: 6385.478478391738
R² Score: -6842.000743669374

Opponent Score Prediction Metrics:
MAE: 67.21358634601742
MSE: 4645.350957545257
R² Score: -6157.347301342567


### Predicting

In [17]:
predict_data = pd.read_csv("basketball_games_data.csv")
predict_data

,Location,Team,Opponent,ADJO,ADJD,EFG%,TO%,OR%,FTR,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,opp_adj_o,opp_adj_d
0,Neutral,Auburn,Purdue,131.0,95.6,59.4,12.9,35.4,29.6,57.3,18.4,29.5,41.4,120.5,100.0
1,Home,Houston,Texas A&M Corpus Christi,119.7,88.6,52.1,14.6,38.0,27.4,54.9,17.8,30.9,40.5,107.0,106.4
2,Away,Duke,Georgia Tech,118.9,88.5,54.0,15.5,34.4,29.9,49.3,16.0,30.3,33.0,107.6,102.3
3,Home,Gonzaga,Bucknell,124.4,95.8,55.7,13.2,33.6,34.9,49.4,20.1,25.9,41.1,97.0,106.9
4,Home,Florida,North Florida,122.6,97.8,54.7,15.2,40.3,32.3,53.6,13.6,29.1,23.3,112.3,114.7
5,Neutral,Kentucky,Ohio State,122.7,98.2,57.2,13.1,31.5,31.6,57.1,15.7,28.2,34.0,114.6,99.7
6,Away,Marquette,Xavier,118.8,96.9,54.1,13.1,31.9,27.2,54.9,16.4,29.2,35.9,113.5,101.5
7,Neutral,UCLA,North Carolina,113.1,92.3,54.0,16.0,35.2,34.4,52.6,14.2,28.6,37.5,116.8,99.6
8,Neutral,Maryland,Syracuse,115.9,95.1,57.4,13.1,33.7,31.4,50.8,14.2,29.4,38.0,112.6,107.3
9,Away,Connecticut,Butler,123.4,101.5,57.9,14.7,35.7,33.5,53.6,18.9,29.9,48.4,113.4,102.9


In [18]:
predict_data["Location"] = predict_data["Location"].replace(
    {"Home": "H", "Away": "A", "Neutral": "N"}
)

In [19]:
predict_data_process = preprocess(predict_data)
predict_data_process = predict_data_process.drop(columns=["Team", "Opponent"])
predict_data_process = predict_data_process.fillna(0)
predict_data_scaled = scaler_X.transform(predict_data_process)

In [20]:
predictions_scaled = model.predict(predict_data_scaled)
predictions = scaler_y.inverse_transform(predictions_scaled)

# Add predictions to the original dataset
predict_data["Predicted_Team_Score"] = predictions[:, 0]
predict_data["Predicted_Opponent_Score"] = predictions[:, 1]

In [21]:
predict_data[["Team", "Opponent", "Predicted_Team_Score", "Predicted_Opponent_Score"]]

,Team,Opponent,Predicted_Team_Score,Predicted_Opponent_Score
0,Auburn,Purdue,93.884184,83.362409
1,Houston,Texas A&M Corpus Christi,78.648988,71.996928
2,Duke,Georgia Tech,79.262610,70.013190
3,Gonzaga,Bucknell,86.535676,66.747275
4,Florida,North Florida,78.274189,70.946764
5,Kentucky,Ohio State,87.114843,83.541679
6,Marquette,Xavier,79.771042,74.228036
7,UCLA,North Carolina,85.701754,83.920037
8,Maryland,Syracuse,84.069536,75.995511
9,Connecticut,Butler,82.303521,74.318529
